In [1]:
import numpy as np

# Regularization

This is a common trick in machine learning, to add a **regularization term** to our loss function to prevent overfitting.

## Logistic Regression

In logistic regression, we can achieve this by using the following loss function $J(W, b)$:

$$J(w,b) = \frac{1}{m} \sum_{i=1}^{m} \mathcal{L} (\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m} \Vert W \Vert_{2}^{2}$$

The L2 norm $\Vert W \Vert_{2}^{2}$ can be expressed as:

$$\Vert W \Vert_{2}^{2} = \sum_{j=1}^{n_x} w_j^2 = W^T W$$

We normally omit $b$ - our bias vector during regularization.

## Neural Network

Similar to logistic regression, we now perform regularization on every layer in our neural network.

$$J(W^{[1]},b^{[1]}, W^{[2]},b^{[2]}, \dots) = \frac{1}{m} \sum_{i=1}^{m} \mathcal{L} (\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m} \sum_{l=1}^{L} \Vert W^{[l]} \Vert^{2}$$

Instead of a L2 norm, $\Vert W \Vert^{2}$ is known as the *Fornebius norm*. We can express it as the following:

$$\Vert W^{[l]} \Vert^{2} = \sum_{i=1}^{n^{[l-1]}} \sum_{j=1}^{n^{[l]}} (W_{ij}^{[l]})^2$$

### Backpropagation

This regularization has to be taken into account during the backprop iteration as well. The `dW` updates now have to include an additional **weight decay**. i.e. the weight decreases in every iteration, controlled by $\lambda$.

$$W^{[l]} := W^{[l]} - \alpha [(\text{term from normal backprop}) + \frac{\lambda}{m} W^{[l]}]$$
$$W^{[l]} := W^{[l]} - \frac{\alpha \lambda}{m} W^{[l]} - \alpha (\text{term from backprop})$$

# Why does Regularization work?

If we let $\lambda$ to be large, we push the value of $W$ closer to 0. This achieves the similar effect as **removing neurons** from the neural network.

# Dropout Regularization

We think of dropout as tossing a coin for each node in a layer. We either keep the node or remove the nodes.

## Implementation

We can achieve this effect by initializing a random matrix in the same shape as the weight matrices and set a threshold value `keep_probs`. We set the values in the matrix as $0$ if it is lesser than `keep_probs`, and $1$ if it is more than or equal to `keep_probs`. By performing an element-wise multiplication between the weight matrix and the dropout matrix, some of the weights will be set to value $0$ - resulting in a 'deactivation' of the hidden unit.

To prevent the expected value of $Z$ from decreasing due to this dropout, we divide the weight matrix by `keep_probs` after applying dropout.

In [4]:
# Sample implementation
keep_prob = 0.8

W = np.random.randn(10, 5)
D = np.random.randn(10, 5) < keep_prob

W = W * D
W /= keep_prob

At test time, we **do not** perform dropout or the renormalization. We simply let the neural net perform feedforward as per normal.

## Intuition

There are formal proofs that show dropout has a similar effect to L2 regularization, but an easier intuition is to see it as we do not want to be too dependent on a single feature. Therefore we need to 'spread' the weights out.

## Downsides of Dropout

By having dropout layers, our cost function $J$ will not be that well defined. When applying this to real world applications, it is safer to turn off dropout first, then adding it in to solve the problem of overfitting.

# Data Augmentation

Another way to add regularization to our model is to augment our data such as:

- Flipping images
- Rotating images
- Crop images

...and others.

This is particulary useful for images, not so much for other kinds of structured data.

# Early Stopping

Another technique to prevent overfitting is 'early stopping'. We first plot $J$ as a function of the number of iterations, and we simply stop training the model once we have stopped improving our loss value.

## Downside of this method

Andrew did mention that, similar to software engineering, we should try and decouple the process of optimizing the minimum cost value, and prevention of overfitting. The method of early stopping couples the two process together, resulting in a less than ideal development proces. There's a concept called **orthoganilization** that will be covered in the later parts of this course.

# Normalizing Inputs

Just like GOML, normalizing inputs helps the convergence of the cost function. This is done in two steps.

1. Subtract mean - $x = x - \mu$
2. Normalize variable - $x = \frac{x}{\sigma}$

It is important to use the same $\mu$ and $\sigma$ for both test/dev and train set. 

Normalization is more useful if we have features on different scales. i.e. feature 1 could be in order of $10^3$, but feature 2 is in order of $10^-3$. Having such input values results in a difficult to navigate shape, hence limiting the effectiveness of our optimization algorithm.